## Instanciating Spark

Import the PySpark library and the `SparkSession` class

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").appName("spark_sql").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/16 12:52:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load downloaded data 

In [3]:
# Reading all the files for each year as well as their respecitve month subfolders for green taxi data
df_green = spark.read.parquet("data/raw/green/*/*")

In [4]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-18 15:52:30|  2019-12-18 15:54:39|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.

In [5]:
df_green.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
# Repeating the same steps for yellow taxi data
df_yellow = spark.read.parquet("data/raw/yellow/*/*")
df_yellow.count()


55553400

## Combine yellow and green taxi datasets

In [7]:
# Change field name before combining
df_green = df_green.withColumnsRenamed({'lpep_pickup_datetime':'pickup_datetime',
                                        'lpep_dropoff_datetime':'dropoff_datetime'})
df_green.show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2|2019-12-18 15:52:30|2019-12-18 15:54:39|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.5|      0.01

In [8]:
# Same thing for yellow taxi dataset
df_yellow = df_yellow.withColumnsRenamed({'tpep_pickup_datetime':'pickup_datetime',
                                          'tpep_dropoff_datetime':'dropoff_datetime'})

df_yellow.show()

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1|2020-01-01 00:28:15|2020-01-01 00:33:03|            1.0|          1.2|       1.0|                 N|         238|         239|           1|        6.0|  3.0|    0.5|      1.47|         0

In [9]:
# First let us identify the similar column fields between both datasets
same_fields = [col for col in df_yellow.columns if col in df_green.columns]

In [10]:
# Alternatively we can use `set(df_green.columns) & set(df_yellow.columns)` to get the same result
same_fields

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [11]:
# We need to filter each dataframe as well as create new column `service_type`
from pyspark.sql import functions as F

df_green_filtered = df_green.select(same_fields).withColumn('service_type', F.lit('green'))
df_yellow_filtered = df_yellow.select(same_fields).withColumn('service_type', F.lit('yellow'))


In [12]:
# Now to combine both datasets
df_trips_data = df_green_filtered.unionAll(df_yellow_filtered)
df_trips_data.show()

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|service_type|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------+
|       2|2019-12-18 15:52:30|2019-12-18 15:54:39|            5.0|          0.0|       1.0|                 N|         264|         264|         1.0|        3.5|  0.5|    0.5|      0.01|       

In [13]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



## Write Spark SQL query

In [14]:
# Convert spark dataframe into a temporary table
df_trips_data.createOrReplaceTempView("fact_trips")

In [15]:
# Test query #1
spark.sql(
    """
SELECT
    *
FROM fact_trips
LIMIT 10;
"""
).show()

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|service_type|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------+
|       2|2019-12-18 15:52:30|2019-12-18 15:54:39|            5.0|          0.0|       1.0|                 N|         264|         264|         1.0|        3.5|  0.5|    0.5|      0.01|       

In [16]:
# Test query #2
spark.sql("""
select
    service_type,
    count(1)
from fact_trips
group by service_type;
"""
).show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [17]:
# SQL query for dm_monthly_zone_revenue.sql

df_result = spark.sql(
    """
    SELECT
        PULocationID AS revenue_zone,
        date_trunc('month', pickup_datetime) AS revenue_month,
        service_type,

        -- Revenue calculation
        SUM(fare_amount) AS revenue_monthly_fare,
        SUM(extra) AS revenue_monthly_extra,
        SUM(mta_tax) AS revenue_monthly_mta_tax,
        SUM(tip_amount) AS revenue_monthly_tip_amount,
        SUM(tolls_amount) AS revenue_monthly_tolls_amount,
        SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
        SUM(total_amount) AS revenue_monthly_total_amount,
        SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

         -- Additional calculations
        AVG(passenger_count) AS avg_monthly_passenger_count,
        AVG(trip_distance) AS avg_monthly_trip_distance
    FROM
        fact_trips
    GROUP BY 1,2,3
    """
)


In [18]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+---------------------------+-------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_monthly_passenger_count|avg_monthly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+---------------------------+-------------------------+
|         232

In [19]:
# Now to write the result into a parquet file
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')